In [1]:
# import libraries
import os
import pandas as pd
from vanna.ollama import Ollama
from vanna.chromadb import ChromaDB_VectorStore

In [2]:
# setup directory and filepath
cdir = os.getcwd()
chroma = os.path.join(cdir, "database")
database = os.path.join(cdir, "database.sqlite")
training = os.path.join(cdir, "training", "dataset.csv")

In [3]:
# instantiate vanna
class Vanna(ChromaDB_VectorStore, Ollama):
    def __init__(self, config=None) -> None:
        ChromaDB_VectorStore.__init__(self, config=config)
        Ollama.__init__(self, config=config)


vn = Vanna(
    config={
        "model": "qwen2.5:3b",
        "path": chroma,
    }
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [4]:
# connect to database
vn.connect_to_sqlite(database)

In [5]:
# training with ddls
ddls = vn.run_sql("SELECT type, sql FROM sqlite_master WHERE sql is not null")
for ddl in ddls["sql"].to_list():
    vn.train(ddl=ddl)

Adding ddl: CREATE TABLE campaign (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    offer_id INTEGER,
    channel TEXT,
    target_segment TEXT,
    start_date TEXT,
    end_date TEXT,
    FOREIGN KEY (offer_id) REFERENCES offer(id)
)


Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given


Adding ddl: CREATE TABLE sqlite_sequence(name,seq)
Adding ddl: CREATE TABLE customer (
    msisdn TEXT PRIMARY KEY,
    name TEXT,
    gender TEXT,
    birth_date TEXT,
    region_id INTEGER,
    FOREIGN KEY (region_id) REFERENCES region(id)
)
Adding ddl: CREATE TABLE offer (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    product_id INTEGER,
    price REAL,
    description TEXT,
    FOREIGN KEY (product_id) REFERENCES product(id)
)
Adding ddl: CREATE TABLE product (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    type TEXT,
    quota_mb REAL,
    price REAL
)
Adding ddl: CREATE TABLE purchase_log (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    msisdn TEXT,
    product_id INTEGER,
    offer_id INTEGER,
    purchase_date TEXT,
    price REAL,
    payment_method TEXT,
    status TEXT,
    FOREIGN KEY (msisdn) REFERENCES customer(msisdn),
    FOREIGN KEY (product_id) REFERENCES product(id),
    FOREIGN KEY (offer_id) REFERENCES offer(id)
)
Adding ddl: CREATE TABL

In [6]:
# training with questions and answer pairs
df = pd.read_csv(training, delimiter=";")

for index, row in df.iterrows():
    question = row["question"]
    answer = row["answer"] + ";"
    vn.train(question=question, sql=answer)

Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given
